In [ ]:
#default_exp cifar_loader

# CIFAR Loader





STATUS: SUPER EARLY ALPHA

This package is NOT yet ready for PUBLIC CONSUMPTION. Use at your own RISK!!!!

Everything, including the API (and even
the existence of this module) are subject
to breaking change...


These are utilities for adapting Pytorch/torchvision Datasets to be used in fastai TPU training.

In [ ]:
#hide
#colab
!pip install -Uqq git+https://github.com/fastai/fastai.git

     |████████████████████████████████| 61kB 6.9MB/s 


In [ ]:
#hide
#colab
!curl -s https://course19.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [ ]:
#hide
!pip freeze | grep torch
!pip freeze | grep fast

torch==1.7.0+cu101
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.1+cu101
fastai==2.2.5
fastcore==1.3.19
fastdtw==0.3.4
fastprogress==1.0.0
fastrlock==0.5


In [ ]:
# hide
# Start of kernel

In [ ]:
#export
def download_torch_dsets(path, torch_dset):
    """Download cifar10 datasets using torchvision utils

       Arguments:
           path (pathlib.Path): path to download the dataset (aka root)
    """
    train_dataset = torch_dset(
        root=path,
        train=True,
        download=True

        )
    test_dataset = torch_dset(
        root=path,
        train=False,
        download=True,
        )
    return train_dataset,test_dataset

In [ ]:
#exporti
import numpy as np
import torchvision.datasets.utils as tv_utils
import torchvision.datasets.cifar as cifar_dsets
import pickle

In [ ]:
#hide
from fastai.data.external import Config
from pathlib import Path
import torchvision.datasets.cifar as cifar_dset
# patch pathlib with fastai goodies
import fastcore.xtras 
cfg = Config()
cifar_root = cfg.data_path/'cifar'
# cifar_root = Path('/tmp/cifar')
cifar_root.mkdir(parents=True,exist_ok=True)
cifar_root.ls()
_ = download_torch_dsets(cifar_root, cifar_dset.CIFAR10)
cifar_root.ls()

Extracting /root/.fastai/data/cifar/cifar-10-python.tar.gz to /root/.fastai/data/cifar
Files already downloaded and verified


(#2) [Path('/root/.fastai/data/cifar/cifar-10-python.tar.gz'),Path('/root/.fastai/data/cifar/cifar-10-batches-py')]

In [ ]:
#export
def load_torch_items(downloaded_list, path, check=False):
    """loads cifar test/train items into tuple(data, target)

       scrobbled together from torch.data.utils.datasets.CIFAR10 code
       https://pytorch.org/docs/stable/_modules/torchvision/datasets/cifar.html#CIFAR10

       Arguments:
           downloaded_list : a list of file names with their checksum, see CIFAR10.train_list or CIFAR10.test_list.
           path (pathlib.Path): the root path where the dataset was downloaded
           check(bool, optional): whether to perform an integrity check on the downloaded files (default: False)
    """
    data = []
    targets = []
    # now load the picked numpy arrays
    for file_name, checksum in downloaded_list:
        file_path = path/cifar_dsets.CIFAR10.base_folder/file_name
        if check and not tv_utils.check_integrity(file_path, checksum):
            raise RuntimeError(
        f'Data checksum failed for file:{file_path} checksum:{checksum}')
        with open(file_path, 'rb') as f:
            entry = pickle.load(f, encoding='latin1')
            data.append(entry['data'])
            if 'labels' in entry:
                targets.extend(entry['labels'])
            else:
                targets.extend(entry['fine_labels'])

    data = np.vstack(data).reshape(-1, 3, 32, 32)
    data = data.transpose((0, 2, 3, 1))  # convert to HWC

    return data, targets

In [ ]:
#hide
tst_data, tst_targets = load_torch_items(cifar_dsets.CIFAR10.test_list, cifar_root, check=True)

In [ ]:
#hide
from fastcore.test import *

In [ ]:
#hide
test_eq(tst_data.shape,(10000,32,32,3))
test_eq(len(tst_targets),10000)
assert isinstance(tst_data,np.ndarray)
assert isinstance(tst_targets[0], int)

In [ ]:
#export
# TODO: incorporate list of classes into dataloaders vocab and decodes

from fastcore.foundation import L
def load_classes(path):
    """Load classes to used to map categories to target labels"""
    base_folder = cifar_dsets.CIFAR10.base_folder
    meta = cifar_dsets.CIFAR10.meta
    file_path = path/base_folder/meta['filename']
    if not tv_utils.check_integrity(file_path, meta['md5']):
        raise RuntimeError('Dataset metadata file not found or corrupted.' +
                           ' You can use download=True to download it')
    data = {}
    with open(file_path, 'rb') as infile:
        data = pickle.load(infile, encoding='latin1')
    classes = data[meta['key']]
#     class_for i, _class in enumerato_idx = {_class: i te(classes)}
    return L(classes)

In [ ]:
#hide
classes = load_classes(cifar_root)

In [ ]:
#hide
test_eq(len(classes),10); classes

(#10) ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
#hide
train_items = load_torch_items(cifar_dsets.CIFAR10.train_list, cifar_root, check=True)

In [ ]:
#exporti
from fastcore.transform import Transform
import torchvision.transforms.functional as TVF
import torch
import PIL

In [ ]:
#export
class CifarNP2ImageTransform(Transform):
    def encodes(self, o:np.ndarray) -> None:
        return PIL.Image.fromarray(o)

In [ ]:
#export
class Int2TensorTransform(Transform):
    def encodes(self, o: int) -> None:
        return torch.tensor(o)

In [ ]:
#export
class CifarImageTransform(Transform):
    def encodes(self, o: PIL.Image) -> None:
        return TVF.to_tensor(o)

In [ ]:
#export
class CifarImage2FloatTransform(Transform):
    def encodes(self, o: torch.Tensor) -> None:
        return o.float().div_(255.)

In [ ]:
#exporti
import torchvision as thv

In [ ]:
#export
def make_torch_tfms():
    norm = thv.transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
    transform_train = thv.transforms.Compose([
        thv.transforms.RandomCrop(32, padding=4),
        thv.transforms.RandomHorizontalFlip(),
        thv.transforms.ToTensor(),
        norm,
    ])
    transform_test = thv.transforms.Compose([
        thv.transforms.ToTensor(),
        norm,
    ])
    return transform_train, transform_test

In [ ]:
th_train_tfms, th_test_tfms = make_torch_tfms()

In [ ]:
#exporti
from fastcore.transform import ItemTransform
from fastcore.basics import store_attr

In [ ]:
#export
class CifarTupleTransform(ItemTransform):
    def __init__(self, x_tfm, y_tfm):
        store_attr()
    def encodes(self,xy):
        return [self.x_tfm(xy[0]), self.y_tfm(xy[1])]

In [ ]:
#exporti
import torch.utils.data as th_data
from torch.utils.data import Dataset

In [ ]:
#export
# TODO: Use TupleTorchDS to create torch dataloaders

class TupleTorchDS(th_data.Dataset):
    def __init__(self, items, x_tfm=None, y_tfm=None):
        store_attr()

    def __len__(self):
        return len(self.items)

    def __getitem__(self, index):
        x,y = self.items[index]
        x = self.x_tfm(x) if self.x_tfm is not None else x
        y = self.y_tfm(y) if self.y_tfm is not None else y
        return (x,y)

In [ ]:
#exporti
from fastcore.transform import Pipeline

In [ ]:
#export
i2t_tfm = Int2TensorTransform() # cnvt int -> torch.tensor
cfnp2img_tfm = CifarNP2ImageTransform() # cnvt ndarray -> PIL.Image
cfimg_tfm = CifarImageTransform() # cnvt PIL.Image -> torch.tensor
cfimg2float_tfm = CifarImage2FloatTransform() # cnvt tensor int -> float + div 255

def make_cifar_item_tfm(th_img_tfms=None):
    img_tfms = [cfnp2img_tfm]
    if th_img_tfms is not None:
        # assumes th_img_tfms incl ToTensor (cnvt2 PIL.Image -> tensor + div by 255)
        img_tfms += [th_img_tfms]
    else:
        img_tfms += [cfimg_tfm, cfimg2float_tfm]

    return CifarTupleTransform(x_tfm=Pipeline(img_tfms), y_tfm=i2t_tfm)

In [ ]:
all_fastai_tfms = make_cifar_item_tfm()
mixed_fastai_train_tfms = make_cifar_item_tfm(th_train_tfms)

In [ ]:
#exporti
from fastai.data.core import TfmdDL
from fastai.data.core import TfmdLists
from fastcore.foundation import L

In [ ]:
#export
def make_cifar_tls(file_list, path, item_tfm, check=True):
    data, targets = load_torch_items(file_list, path, check=check)
    item_tuples = L(data,targets).zip()
    tls = TfmdLists(item_tuples,[item_tfm])
    return tls

In [ ]:
#export
def make_cifar_dl(file_list, path, th_img_tfms=None, check=True, bs=64, **kwargs):
    item_tfm = make_cifar_item_tfm(th_img_tfms)
    tls = make_cifar_tls(file_list, path, item_tfm, check=check)
    dl = TfmdDL(tls,bs=bs, **kwargs)
    return dl

In [ ]:
train_tls = make_cifar_tls(cifar_dsets.CIFAR10.train_list,
                           cifar_root,mixed_fastai_train_tfms)

In [ ]:
train_tls.tfms[0].x_tfm[1]

Compose:
encodes: (object,object) -> Composedecodes: 

In [ ]:
train_dl1 = make_cifar_dl(cifar_dsets.CIFAR10.train_list, cifar_root)

In [ ]:
#exporti 
from fastai.data.core import DataLoaders

In [ ]:
#export
def make_fastai_cifar_dls(path, bs=64, check=True, device=None, **kwargs):
    train_tfm, test_tfm = make_torch_tfms()
    train_dl = make_cifar_dl(
        cifar_dsets.CIFAR10.train_list,
        path,
        train_tfm,
        check=check, bs=bs,
        shuffle=True)
    test_dl = make_cifar_dl(
        cifar_dsets.CIFAR10.test_list,
        path,
        test_tfm,
        check=check, bs=bs,
        shuffle=False)
    dls = DataLoaders(train_dl, test_dl, device=device)
    return dls

In [ ]:
import torch.cuda
import torch
device = torch.device(torch.cuda.current_device()) if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [ ]:
cifar_dls = make_fastai_cifar_dls(cifar_root, device=device)

In [ ]:
(cifar_root/'cifar-10-batches-py').ls()

(#8) [Path('/root/.fastai/data/cifar/cifar-10-batches-py/data_batch_1'),Path('/root/.fastai/data/cifar/cifar-10-batches-py/data_batch_2'),Path('/root/.fastai/data/cifar/cifar-10-batches-py/readme.html'),Path('/root/.fastai/data/cifar/cifar-10-batches-py/batches.meta'),Path('/root/.fastai/data/cifar/cifar-10-batches-py/test_batch'),Path('/root/.fastai/data/cifar/cifar-10-batches-py/data_batch_4'),Path('/root/.fastai/data/cifar/cifar-10-batches-py/data_batch_5'),Path('/root/.fastai/data/cifar/cifar-10-batches-py/data_batch_3')]

In [ ]:
cifar_dsets.CIFAR10.train_list

[['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
 ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
 ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
 ['data_batch_4', '634d18415352ddfa80567beed471001a'],
 ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb']]

In [ ]:
xb, yb = cifar_dls.one_batch()

In [ ]:
xb.dtype

torch.float32

In [ ]:
xb.shape

torch.Size([64, 3, 32, 32])

In [ ]:
yb.dtype

torch.int64

In [ ]:
yb.shape

torch.Size([64])

In [ ]:
xb.device

device(type='cuda', index=0)

In [ ]:
yb.device

device(type='cuda', index=0)

In [ ]:
cifar_dls.train.after_batch

Pipeline: 

In [ ]:
type(cifar_dls.train.dataset)

fastai.data.core.TfmdLists

In [ ]:
hasattr(cifar_dls, 'device')

True

In [ ]:
cifar_dls.device is None

False

In [ ]:
cifar_dls.path

Path('.')

In [ ]:
hasattr(cifar_dls.loaders[0],'to')

True

In [ ]:
cifar_dls.loaders[0].device is None

False

In [ ]:
from fastai.vision.all import *
import fastai.callback.progress

In [ ]:
from fastai.vision.models import resnet18

In [ ]:
from fastai.metrics import accuracy

In [ ]:
import torch.nn as nn

In [ ]:
learner = cnn_learner(cifar_dls, resnet18, n_out=10, pretrained=False, 
                      normalize=False,
                      loss_func=nn.CrossEntropyLoss(),metrics=accuracy)

In [ ]:
learner.show_training_loop()

Start Fit
   - before_fit     : [TrainEvalCallback, Recorder, ProgressCallback]
  Start Epoch Loop
     - before_epoch   : [Recorder, ProgressCallback]
    Start Train
       - before_train   : [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - before_batch   : []
         - after_pred     : []
         - after_loss     : []
         - before_backward: []
         - before_step    : []
         - after_step     : []
         - after_cancel_batch: []
         - after_batch    : [TrainEvalCallback, Recorder, ProgressCallback]
      End Batch Loop
    End Train
     - after_cancel_train: [Recorder]
     - after_train    : [Recorder, ProgressCallback]
    Start Valid
       - before_validate: [TrainEvalCallback, Recorder, ProgressCallback]
      Start Batch Loop
         - **CBs same as train batch**: []
      End Batch Loop
    End Valid
     - after_cancel_validate: [Recorder]
     - after_validate : [Recorder, ProgressCallback]
  End Epoch Loop
   - after_

In [ ]:
learner.summary()

Sequential (Input shape: 64)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 64 x 16 x 16   
Conv2d                                    9408       True      
BatchNorm2d                               128        True      
ReLU                                                           
MaxPool2d                                                      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
Conv2d                                    36864      True      
BatchNorm2d                               128        True      
ReLU                                                           
Conv2d                                    36864      True      
BatchNorm2d                      

In [ ]:
#colab
%%time
learner.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,1.889744,1.896039,0.365500,00:51
1,1.525709,1.402483,0.493500,00:52
2,1.160433,1.045405,0.641900,00:51
3,1.002535,0.890528,0.687900,00:51
4,0.874758,0.821122,0.709400,00:51


CPU times: user 2min 38s, sys: 3.3 s, total: 2min 42s
Wall time: 4min 18s
